# Main Work Notebook

In [ ]:
# System audio I/O backend
# !sudo apt-get update && sudo apt-get install -y ffmpeg

# Python deps
!pip install silero-vad

[sudo] password for sharjeel: 
sudo: a password is required
^C
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 1.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 2.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [silero-vad]5 [onnxruntime]


In [3]:
!poetry run python trim_with_silero_vad.py \
  --in_dir ./Duaa \
  --out_dir wavs \
  --keep_silence 0.25 \
  --energy_threshold 0.4 \
  --min_speech_ms 80 \
  --min_silence_ms 200 \
  --viz_dir data/vizNew \
  --vad_dir data/vad_json


ok          Female_ID_41457967_10.wav
ok          Female_ID_41457967_103.wav
ok          Female_ID_41457967_107.wav
ok          Female_ID_41457967_11.wav
ok          Female_ID_41457967_12.wav
ok          Female_ID_41457967_13.wav
ok          Female_ID_41457967_15.wav
ok          Female_ID_41457967_16.wav
ok          Female_ID_41457967_17.wav
ok          Female_ID_41457967_172.wav
ok          Female_ID_41457967_173.wav
ok          Female_ID_41457967_174.wav
ok          Female_ID_41457967_175.wav
ok          Female_ID_41457967_176.wav
ok          Female_ID_41457967_177.wav
ok          Female_ID_41457967_178.wav
ok          Female_ID_41457967_179.wav
ok          Female_ID_41457967_18.wav
ok          Female_ID_41457967_180.wav
ok          Female_ID_41457967_181.wav
ok          Female_ID_41457967_182.wav
ok          Female_ID_41457967_183.wav
ok          Female_ID_41457967_184.wav
ok          Female_ID_41457967_185.wav
ok          Female_ID_41457967_186.wav
ok          Female_ID_41457967_18

## Working on the Excel File

In [1]:
import pandas as pd

excel_file_path = './Egyptian Arabic TTS Duaa.xlsx'
df = pd.read_excel(excel_file_path, header=None, names=['fileName', 'transcription', 'status'])

display(df.head())

,fileName,transcription,status
0,a-Female_ID_41457969_1,أخبارَك إيه؟,OLD
1,a-Female_ID_41457969_2,أُختَك كوَيِّسَه؟,OLD
2,a-Female_ID_41457969_3,أرْخَص شويَّة.,OLD
3,a-Female_ID_41457969_4,أُسْتَاذ كِبِير.,OLD
4,a-Female_ID_41457969_5,مَالوش أَصْل,OLD


In [2]:
import os
from pathlib import Path

# 1) Load Excel (you already did this)
excel_file_path = './Egyptian Arabic TTS Duaa.xlsx'
df = pd.read_excel(excel_file_path, header=None, names=['fileName', 'transcription', 'status'])

# rename column
df = df.rename(columns={'transcription': 'text'})

# Normalize Excel filenames: ensure they have .wav extension
df['fileName_norm'] = df['fileName'].astype(str).str.strip()
df['fileName_norm'] = df['fileName_norm'].apply(
    lambda x: x if x.lower().endswith('.wav') else x + '.wav'
)

# 2) Collect all wav filenames from wavs/ (recursive)
wavs_root = Path('wavs')
wav_files = [p.name for p in wavs_root.rglob('*.wav')]  # just basenames

wav_set = set(wav_files)
excel_set = set(df['fileName_norm'])

# 3) Common and missing

# Common: exist in both Excel and wavs folder
common_names = sorted(excel_set & wav_set)

# In Excel but no corresponding wav file
excel_only = sorted(excel_set - wav_set)

# In wavs folder but not listed in Excel
wavs_only = sorted(wav_set - excel_set)

print(f"Common (in both): {len(common_names)}")
print(f"Excel only (no wav): {len(excel_only)}")
print(f"Wavs only (no Excel row): {len(wavs_only)}")

# Optional: create DataFrames for inspection
df_common = df[df['fileName_norm'].isin(common_names)].copy()
df_excel_only = df[df['fileName_norm'].isin(excel_only)].copy()
df_wavs_only = pd.DataFrame({'fileName': wavs_only})


display(df_common.head())
display(df_excel_only.head())
display(df_wavs_only.head())

Common (in both): 1414
Excel only (no wav): 23
Wavs only (no Excel row): 314


,fileName,text,status,fileName_norm
0,a-Female_ID_41457969_1,أخبارَك إيه؟,OLD,a-Female_ID_41457969_1.wav
1,a-Female_ID_41457969_2,أُختَك كوَيِّسَه؟,OLD,a-Female_ID_41457969_2.wav
2,a-Female_ID_41457969_3,أرْخَص شويَّة.,OLD,a-Female_ID_41457969_3.wav
3,a-Female_ID_41457969_4,أُسْتَاذ كِبِير.,OLD,a-Female_ID_41457969_4.wav
4,a-Female_ID_41457969_5,مَالوش أَصْل,OLD,a-Female_ID_41457969_5.wav


,fileName,text,status,fileName_norm
41,a-Female_ID_41457969_42,أنا سِمعت.,OLD,a-Female_ID_41457969_42.wav
143,a-Female_ID_41457969_145,إنتَ فاضي بُكره؟,OLD,a-Female_ID_41457969_145.wav
241,a-Female_ID_41457969_243,التِّليفون رَن ومَعرَفتش أَرُد عَليه عَشان كُن...,OLD,a-Female_ID_41457969_243.wav
275,a-Female_ID_41457969_277,الكُورة اللي امُبَارَح كانت حَماس، والفرِيق ال...,OLD,a-Female_ID_41457969_277.wav
276,a-Female_ID_41457969_278,اللَّحظَة اللِّي اِتقَال فيها الكَلِمَة كَانَت...,OLD,a-Female_ID_41457969_278.wav


,fileName
0,Female_ID_41457967_10.wav
1,Female_ID_41457967_103.wav
2,Female_ID_41457967_107.wav
3,Female_ID_41457967_11.wav
4,Female_ID_41457967_12.wav


### Duplication Identification and Removal

In [3]:
# How many rows vs unique filenames?
print("rows in df_common:", len(df_common))
print("unique fileName_norm:", df_common['fileName_norm'].nunique())

# See duplicated entries by fileName_norm
dups = df_common[df_common.duplicated('fileName_norm', keep=False)].sort_values('fileName_norm')
print("duplicated fileName_norm rows:", len(dups))
display(dups.head(20))

rows in df_common: 1495
unique fileName_norm: 1414
duplicated fileName_norm rows: 162


,fileName,text,status,fileName_norm
1029,Female_ID_41457969_766,عَزِّلنا الشَّقَّة دي من حوالي تَلات سنين.,NaN,Female_ID_41457969_766.wav
1252,Female_ID_41457969_766,عَزِّلنا الشَّقَّة دي من حوالي تَلات سنين.,NaN,Female_ID_41457969_766.wav
1030,Female_ID_41457969_767,لَمّا نِعزِل من هنا إبقَى إعزِم الناس في الشَّ...,NaN,Female_ID_41457969_767.wav
1253,Female_ID_41457969_767,لَمّا نِعزِل من هنا إبقَى إعزِم الناس في الشَّ...,NaN,Female_ID_41457969_767.wav
1254,Female_ID_41457969_768,بِنِعزِّل الأيّام دي وعِندي مِليون حاجة عايزة ...,NaN,Female_ID_41457969_768.wav
1031,Female_ID_41457969_768,بِنِعزِّل الأيّام دي وعِندي مِليون حاجة عايزة ...,NaN,Female_ID_41457969_768.wav
1032,Female_ID_41457969_769,مِش هَأعزِل من هنا، البيت ده فيه كُلّ ذِكرياتي.,NaN,Female_ID_41457969_769.wav
1255,Female_ID_41457969_769,مِش هَأعزِل من هنا، البيت ده فيه كُلّ ذِكرياتي.,NaN,Female_ID_41457969_769.wav
1033,Female_ID_41457969_770,عَزِّلوا بَقى من الحِتّة دي، دي بَقت وِحشة أوي.,NaN,Female_ID_41457969_770.wav
1256,Female_ID_41457969_770,عَزِّلوا بَقى من الحِتّة دي، دي بَقت وِحشة أوي.,NaN,Female_ID_41457969_770.wav


In [4]:
df_common_nodup = (
    df_common
    .sort_values('fileName_norm')          # optional, stable order
    .drop_duplicates(subset='fileName_norm', keep='first')
    .reset_index(drop=True)
)

print("after dedup:", len(df_common_nodup))

after dedup: 1414


### Phonemization

In [5]:
from tqdm import tqdm
from phonemizer.backend import EspeakBackend
from phonemizer.punctuation import Punctuation

# Create backend once
backend = EspeakBackend(language='ar', preserve_punctuation=True,  with_stress=True)
# punct = Punctuation(';:,.!"?()-')
punct = Punctuation("':()\"")  # Only remove ', :, (, ), and "

def phonemize_arabic_text_fast(text):
    text = punct.remove(text)
    return backend.phonemize([text], strip=True)[0]

# Use tqdm with apply for progress bar
df_final = df_common_nodup.copy()
df_final['phonetic_text'] = list(
    tqdm(
        df_final['text'].apply(phonemize_arabic_text_fast),
        total=len(df_final),
        desc="Phonemizing df text"
    )
)

# Preview
print(df_final[['text', 'phonetic_text']].head())


Phonemizing df text: 100%|██████████| 1414/1414 [00:00<00:00, 2910081.38it/s]

                                                text  \
0       المَسَافة مَش سَاعات، المَسَافة فِي القُلوب.   
1    المَسْرَحِيَّه إلّي شُفناها كات مُضحِكه جِدًّا.   
2  الوَقت عَدّى سَريع أوي، ما حَسّيتش إن الشَّهر ...   
3       أنا بِحِسّ إنِّي قُوِية لَمَّا لَقيك جَنبِي.   
4                الفُلُوس اِتْجَمَّعِت فِي الحِسَاب.   

                                       phonetic_text  
0  ʔalmˈasaːft mˈaʃ saːʕˌaːt ʔalmˈasaːft fiː ʔalq...  
1  ʔalmasrˈaħiːˌah ʔllˈi ʃˈufnaːhˌaː kaːt mˈudˤħi...  
2  ʔalwˈaqt ʕˈadːa sarˌiːʕ ʔˈawi mˈaː ħˈassitʃ ʔˈ...  
3  ˈana biħˌiss ʔnnˈiː qˈuːiːt lammˌaː laqˌika dʒ...  
4          ʔalfulˌuːs ʔˌitdʒammˈaʕit fiː ʔalħˈisaːb.  


In [6]:
# Remove columns we don't need in the final table
df_final = df_final.drop(columns=['status', 'fileName_norm'], errors='ignore')

print(len(df_final))
df_final

1414


,fileName,text,phonetic_text
0,Female_ID_41457969_1-0,المَسَافة مَش سَاعات، المَسَافة فِي القُلوب.,ʔalmˈasaːft mˈaʃ saːʕˌaːt ʔalmˈasaːft fiː ʔalq...
1,Female_ID_41457969_1,المَسْرَحِيَّه إلّي شُفناها كات مُضحِكه جِدًّا.,ʔalmasrˈaħiːˌah ʔllˈi ʃˈufnaːhˌaː kaːt mˈudˤħi...
2,Female_ID_41457969_10,الوَقت عَدّى سَريع أوي، ما حَسّيتش إن الشَّهر ...,ʔalwˈaqt ʕˈadːa sarˌiːʕ ʔˈawi mˈaː ħˈassitʃ ʔˈ...
3,Female_ID_41457969_100,أنا بِحِسّ إنِّي قُوِية لَمَّا لَقيك جَنبِي.,ˈana biħˌiss ʔnnˈiː qˈuːiːt lammˌaː laqˌika dʒ...
4,Female_ID_41457969_1000,الفُلُوس اِتْجَمَّعِت فِي الحِسَاب.,ʔalfulˌuːs ʔˌitdʒammˈaʕit fiː ʔalħˈisaːb.
...,...,...,...
1409,a-Female_ID_41457969_97,أنا مِش فاهم إللي بْيحصَل.,ˈana mˈiʃ fˈaːhim ʔllˈiː bjħs̪ˈa.l.
1410,a-Female_ID_41457969_98,أنا مش فاهم النُقطه.,ˈana mˈaʃʃ fˈaːhim ʔannˈuqt̪h.
1411,a-Female_ID_41457969_99,أنا مش قادِر أستَحمِل.,ˈana mˈaʃʃ qˈaːdir ʔstˈaħmil.
1412,aFemale_ID_41457969_1082,العَرَبِيَّه اِتْمَسَحِت وَبَقِت نَضِيفَه.,ʔalʕarˈabiːˌah ʔˌitmasˈaħit wabˈaqit nˈadˤi.ːf...


### File Duration Column

In [7]:
import soundfile as sf

def get_wav_duration(file_path):
  """
  Calculates the duration of a WAV file.

  Args:
    file_path: The path to the WAV file.

  Returns:
    The duration of the WAV file in seconds, or None if an error occurs.
  """
  try:
    info = sf.info(file_path)
    return info.duration
  except Exception as e:
    print(f"Error reading file {file_path}: {e}")
    return None

import os
from pathlib import Path

current_directory = Path('./wavs')
file_list_pathlib = [item.name for item in current_directory.iterdir() if item.is_file()]


file_durations = []


for file_name in file_list_pathlib:
  full_file_path = current_directory / file_name
  duration = get_wav_duration(full_file_path)
  if duration is not None:
    file_durations.append({'fileName': file_name, 'duration': duration})

print(file_durations[:5]) # Print first 5 to check

[{'fileName': 'a-Female_ID_41457969_1.wav', 'duration': 1.6}, {'fileName': 'a-Female_ID_41457969_10.wav', 'duration': 1.9}, {'fileName': 'a-Female_ID_41457969_100.wav', 'duration': 1.9}, {'fileName': 'a-Female_ID_41457969_101.wav', 'duration': 2.4}, {'fileName': 'a-Female_ID_41457969_102.wav', 'duration': 1.8}]


In [8]:
# 1) DataFrame from durations
df_durations = pd.DataFrame(file_durations)  # columns: fileName, duration

# 2) Remove ".wav" from df_durations fileName
df_durations['fileName'] = (
    df_durations['fileName']
    .astype(str)
    .str.strip()
    .str.replace('.wav', '', case=False, regex=False)
)

# 3) Normalize df_final fileName in the same way (remove .wav)
df_final = df_final.copy()
df_final['fileName'] = (
    df_final['fileName']
    .astype(str)
    .str.strip()
    .str.replace('.wav', '', case=False, regex=False)
)

# 4) Merge on fileName without extension
df_final = df_final.merge(
    df_durations,
    on='fileName',
    how='left'
)

df_final

,fileName,text,phonetic_text,duration
0,Female_ID_41457969_1-0,المَسَافة مَش سَاعات، المَسَافة فِي القُلوب.,ʔalmˈasaːft mˈaʃ saːʕˌaːt ʔalmˈasaːft fiː ʔalq...,3.50
1,Female_ID_41457969_1,المَسْرَحِيَّه إلّي شُفناها كات مُضحِكه جِدًّا.,ʔalmasrˈaħiːˌah ʔllˈi ʃˈufnaːhˌaː kaːt mˈudˤħi...,4.70
2,Female_ID_41457969_10,الوَقت عَدّى سَريع أوي، ما حَسّيتش إن الشَّهر ...,ʔalwˈaqt ʕˈadːa sarˌiːʕ ʔˈawi mˈaː ħˈassitʃ ʔˈ...,4.00
3,Female_ID_41457969_100,أنا بِحِسّ إنِّي قُوِية لَمَّا لَقيك جَنبِي.,ˈana biħˌiss ʔnnˈiː qˈuːiːt lammˌaː laqˌika dʒ...,3.20
4,Female_ID_41457969_1000,الفُلُوس اِتْجَمَّعِت فِي الحِسَاب.,ʔalfulˌuːs ʔˌitdʒammˈaʕit fiː ʔalħˈisaːb.,2.75
...,...,...,...,...
1409,a-Female_ID_41457969_97,أنا مِش فاهم إللي بْيحصَل.,ˈana mˈiʃ fˈaːhim ʔllˈiː bjħs̪ˈa.l.,2.10
1410,a-Female_ID_41457969_98,أنا مش فاهم النُقطه.,ˈana mˈaʃʃ fˈaːhim ʔannˈuqt̪h.,2.00
1411,a-Female_ID_41457969_99,أنا مش قادِر أستَحمِل.,ˈana mˈaʃʃ qˈaːdir ʔstˈaħmil.,2.10
1412,aFemale_ID_41457969_1082,العَرَبِيَّه اِتْمَسَحِت وَبَقِت نَضِيفَه.,ʔalʕarˈabiːˌah ʔˌitmasˈaħit wabˈaqit nˈadˤi.ːf...,2.46


### Hugging Dataset Creation + Uploading to repo

In [29]:
from pathlib import Path
from datasets import Dataset, DatasetDict, Audio
from tqdm import tqdm

# Configurable constants
OUTPUT_DIR = Path("wavs")
SAMPLING_RATE = 24000
DEFAULT_GENDER = "female"

def build_hf_row_from_wav(row, output_dir: Path) -> dict:
    """
    Build a Hugging Face dataset row from a dataframe row.
    
    Args:
        row: A pandas row (from itertuples).
        output_dir: Directory containing .wav files.
    
    Returns:
        dict: A single dataset row with metadata.
    """
    wav_path = output_dir / f"{row.fileName}.wav"
    return {
        "text": row.text,
        "phonetic_text": row.phonetic_text,
        "file": row.fileName,
        "audio": str(wav_path),
        "gender": DEFAULT_GENDER,
        "sampling_rate": SAMPLING_RATE,
        "duration": row.duration,
    }

# Column order for consistency
COLUMN_ORDER = ["text", "phonetic_text", "file", "audio", "gender", "sampling_rate", "duration"]

# Build rows efficiently
rows = [
    build_hf_row_from_wav(row, OUTPUT_DIR)
    for row in tqdm(df_final.itertuples(index=False), total=len(df_final), desc="Building dataset")
]

# Create HF dataset and cast audio column
full_ds = Dataset.from_list([{k: r[k] for k in COLUMN_ORDER} for r in rows]).cast_column(
    "audio", Audio(sampling_rate=SAMPLING_RATE, decode=False)
)

# Train/test split
splits = full_ds.train_test_split(test_size=0.1, seed=42)
final_ds = DatasetDict(splits)

# Quick inspection
print(final_ds)
print(final_ds["train"][0])

Building dataset: 100%|██████████| 1414/1414 [00:00<00:00, 129801.18it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'phonetic_text', 'file', 'audio', 'gender', 'sampling_rate', 'duration'],
        num_rows: 1272
    })
    test: Dataset({
        features: ['text', 'phonetic_text', 'file', 'audio', 'gender', 'sampling_rate', 'duration'],
        num_rows: 142
    })
})
{'text': 'دي أُوضتي، فيها شِبّاك كِبير ومِروَحَة جَميلَة.', 'phonetic_text': 'dˈiː ʔˈuːdˤtiː fˈiːhaː ʃˈibːaːkˌa kibˌiːr wmiruːˈaħat dʒamˈiːlat.', 'file': 'Female_ID_41457969_774', 'audio': {'bytes': None, 'path': 'wavs/Female_ID_41457969_774.wav'}, 'gender': 'female', 'sampling_rate': 24000, 'duration': 3.98}


In [27]:
from huggingface_hub import notebook_login
from datasets import DatasetDict

# Log in to Hugging Face Hub (will prompt for your token)
notebook_login()

# Push the dataset to your Hugging Face account
final_ds.push_to_hub("blits-ai/arabic-tts-wav-24k")

Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00,  6.79ba/s]
Processing Files (1 / 1): 100%|██████████|  189MB /  189MB, 4.69MB/s  
New Data Upload: 100%|██████████|  189MB /  189MB, 4.69MB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 17.79ba/s]
Processing Files (1 / 1): 100%|██████████| 19.6MB / 19.6MB, 2.88MB/s  
New Data Upload: 100%|██████████| 19.6MB / 19.6MB, 2.88MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:08<00:00,  8.69s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/blits-ai/arabic-tts-wav-24k/commit/c15883a876565a722a35ab2df754a113280d345f', commit_message='Upload dataset', commit_description='', oid='c15883a876565a722a35ab2df754a113280d345f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/blits-ai/arabic-tts-wav-24k', endpoint='https://huggingface.co', repo_type='dataset', repo_id='blits-ai/arabic-tts-wav-24k'), pr_revision=None, pr_num=None)